# Image Classification using tf.keras

### Importing Packages

In [0]:
!pip install tf-nightly-gpu
!pip install "tensorflow_hub==0.4.0"
!pip install -U tensorflow_datasets

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import numpy as np
import glob
import shutil
import matplotlib.pyplot as plt

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
tf.enable_eager_execution()

import tensorflow_hub as hub
import tensorflow_datasets as tfds

from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Load Data


Dowload flowers dataset from google and join to path


In [0]:
_URL = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"

zip_file = tf.keras.utils.get_file(origin=_URL, 
                                   fname="flower_photos.tgz", 
                                   extract=True)

base_dir = os.path.join(os.path.dirname(zip_file), 'flower_photos')

Create classes for flowers


In [0]:
classes = ['roses', 'daisy', 'dandelion', 'sunflowers', 'tulips']

Loop through classes and slice each class into train and validation sets

In [0]:
for cl in classes:
  img_path = os.path.join(base_dir, cl)
  images = glob.glob(img_path + '/*.jpg')
  print("{}: {} Images".format(cl, len(images)))
  
  # slice the dataset at 70% and put them into training and validation sets
  train, val = images[:round(len(images)*0.7)], images[round(len(images)*0.7):]

  # make dir and then move images into
  for t in train:
    if not os.path.exists(os.path.join(base_dir, 'train', cl)):
      os.makedirs(os.path.join(base_dir, 'train', cl))
    shutil.move(t, os.path.join(base_dir, 'train', cl))

  for v in val:
    if not os.path.exists(os.path.join(base_dir, 'val', cl)):
      os.makedirs(os.path.join(base_dir, 'val', cl))
    shutil.move(v, os.path.join(base_dir, 'val', cl))

Set up paths for train and val

In [0]:
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')

### Data Augmentation

Set batch size and img shape to input into model

In [0]:
BATCH_SIZE = 100
IMG_SHAPE = 224

Create geterator that instructs to: rescale images to 255 and add in random rotation, zoom, horizontal flip, width shift and height shift. We also add in shuffle. 

This helps to train the model on a more difficult dataset - building a more rohbust model that will not over fit as much.

In [0]:
image_gen_train = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      zoom_range=0.3,
      horizontal_flip=True,
      fill_mode='nearest')

train_data_gen = image_gen_train.flow_from_directory(
      batch_size=BATCH_SIZE,
      directory=train_dir,
      shuffle=True,
      target_size=(IMG_SHAPE,IMG_SHAPE),
      class_mode='sparse')

In [0]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
    plt.tight_layout()
    plt.show()

Visualise these augmentations for the first 5 images

In [0]:
augmented_images = [train_data_gen[0][0][5] for i in range(5)]
plotImages(augmented_images)

Validation set, because this set plays not part in the training - only validation - we only need to resize the images to fit in the model. There is no need to shuffle and augment the data.


In [0]:
image_gen_val = ImageDataGenerator(rescale=1./255)

val_data_gen = image_gen_train.flow_from_directory(
      batch_size=BATCH_SIZE,
      directory=val_dir,
      target_size=(IMG_SHAPE,IMG_SHAPE),
      class_mode='sparse')

### Create the transfer learned model
 
 Download the pre mobilenet V2 from google tfhub. Initialise the model with keras and then add in a 5 node softmax layer to predict into the 5 classes

In [0]:
# tried to use feature vector
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
feature_extractor = hub.KerasLayer(URL, input_shape=(IMG_SHAPE, IMG_SHAPE, 3)) 

feature_extractor.trainable = False

model = tf.keras.Sequential([
    feature_extractor,
    tf.keras.layers.Dense(5, activation='softmax')
])

model.summary()

### Compile the Model

As normal use the adam optimiser & crossentropy loss

In [0]:
# Compile the model
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## Train the Model



In [0]:
EPOCHS = 120

history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=int(np.ceil(len(os.listdir(train_dir)) / float(BATCH_SIZE))),
    epochs=EPOCHS,
    validation_data=val_data_gen,
    validation_steps=int(np.ceil(len(os.listdir(val_dir)) / float(BATCH_SIZE)))
)


## Plot Training and Validation Graphs.



In [0]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.savefig('./foo.png')
plt.show()